# Bike Stores Relational Database 

In [0]:
pip install kagglehub

In [0]:
%restart_python

In [0]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dillonmyrick/bike-store-sample-database")

print("Path to dataset files:", path)

In [0]:
import os

dataset_path = "/home/spark-99100516-4c2b-4fad-962a-23/.cache/kagglehub/datasets/dillonmyrick/bike-store-sample-database/versions/3"
files = os.listdir(dataset_path)
display(files)

In [0]:
import os

catalog = "learning"
schema = "data"
dataset_path = "/home/spark-99100516-4c2b-4fad-962a-23/.cache/kagglehub/datasets/dillonmyrick/bike-store-sample-database/versions/3"

files = [f for f in os.listdir(dataset_path) if f.endswith(".csv")]

for file in files:
    file_path = os.path.join(dataset_path, file)
    table_name = os.path.splitext(file)[0].replace(" ", "_").lower()
    full_table_name = f"{catalog}.{schema}.{table_name}"
    df = spark.read.option("header", "true").option("inferSchema", "true").csv(file_path)
    df.write.mode("overwrite").saveAsTable(full_table_name)

In [0]:
import os

# Replace with the actual path from the previous cell
dataset_path = "/home/spark-99100516-4c2b-4fad-962a-23/.cache/kagglehub/datasets/dillonmyrick/bike-store-sample-database/versions/3"

# List all files in the dataset directory
files = [f for f in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, f))]

for file in files:
    file_path = os.path.join(dataset_path, file)
    table_name = os.path.splitext(file)[0].replace(" ", "_").lower()
    if file.endswith(".csv"):
        df = spark.read.option("header", "true").option("inferSchema", "true").csv(file_path)
        df.write.mode("overwrite").saveAsTable(table_name)
    elif file.endswith(".parquet"):
        df = spark.read.parquet(file_path)
        df.write.mode("overwrite").saveAsTable(table_name)

In [0]:
import os

catalog = "learning"
schema = "bike_store"
dataset_path = "/Volumes/learning/data/raw/bike-store-sample-database/"

files = [f for f in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, f))]

for file in files:
    file_path = os.path.join(dataset_path, file)
    table_name = os.path.splitext(file)[0].replace(" ", "_").lower()
    full_table_name = f"{catalog}.{schema}.{table_name}"
    if file.endswith(".csv"):
        df = spark.read.option("header", "true").option("inferSchema", "true").csv(file_path)
        df.write.mode("overwrite").saveAsTable(full_table_name)
    elif file.endswith(".parquet"):
        df = spark.read.parquet(file_path)
        df.write.mode("overwrite").saveAsTable(full_table_name)

# Exploratory Data Analysis of all tables in Bike Stores dataset 


In [0]:
tables = [row.tableName for row in spark.sql("SHOW TABLES IN learning.bike_store").collect()]

for table in tables:
    print(f"--- {table} ---")
    df = spark.table(f"learning.bike_store.{table}")
    print("Schema:")
    df.printSchema()
    print("Row count:")
    display(spark.sql(f"SELECT COUNT(*) AS row_count FROM learning.data.{table}"))
    print("Sample rows:")
    display(df.limit(10))
    print("Summary statistics (numeric columns):")
    display(df.describe())
    print("\n")